<a href="https://colab.research.google.com/github/Kasper-Raupach-Haurum/M6-Data-Engineering-and-MLOps/blob/main/M6_Data_Engineering_and_MLOps_Assignment_3_MLflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone "https://github.com/Kasper-Raupach-Haurum/M6-Data-Engineering-and-MLOps.git"

Cloning into 'M6-Data-Engineering-and-MLOps'...
remote: Enumerating objects: 160, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 160 (delta 38), reused 0 (delta 0), pack-reused 107
Receiving objects: 100% (160/160), 343.26 KiB | 777.00 KiB/s, done.
Resolving deltas: 100% (96/96), done.


## **Data library & loading**

At the first phase of the project importing important libraries and loading the HR dataset is the main key. Here, HR dataset from our previous project will be used for Elastic net and it will be transfered to MLflow to track and monitor the project. Where the optimal and best case model can be identified.

In [ ]:
!pip install mlflow --quiet
!pip install pyngrok --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!mlflow

Usage: mlflow [OPTIONS] COMMAND [ARGS]...

Options:
  --version  Show the version and exit.
  --help     Show this message and exit.

Commands:
  artifacts    Upload, list, and download...
  db           Commands for managing an...
  deployments  Deploy MLflow models to custom...
  doctor       Prints out useful information for
               debugging issues with MLflow.
  experiments  Manage experiments.
  gc           Permanently delete runs in the
               `deleted` lifecycle stage.
  models       Deploy MLflow models locally.
  recipes      Run MLflow Recipes and inspect...
  run          Run an MLflow project from the...
  runs         Manage runs.
  sagemaker    Serve models on SageMaker.
  server       Run the MLflow tracking server.


In [ ]:
import os
import shutil
import pprint
import sys
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np

from random import random, randint
import mlflow
import mlflow.sklearn
mlflow.set_experiment('HR_Attrition_1')

from mlflow import log_metric, log_param, log_artifacts
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from mlflow.tracking import MlflowClient
import warnings

2023/04/26 20:30:05 INFO mlflow.tracking.fluent: Experiment with name 'HR_Attrition_1' does not exist. Creating a new experiment.


In [ ]:
warnings.filterwarnings("ignore")
print(mlflow.__version__)

2.3.0


In [ ]:
path = '/content/M6-Data-Engineering-and-MLOps/general_data.csv'

In [ ]:
HR = pd.read_csv(path)

In [ ]:
HR.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      4410 non-null   int64  
 1   Attrition                4410 non-null   object 
 2   BusinessTravel           4410 non-null   object 
 3   Department               4410 non-null   object 
 4   DistanceFromHome         4410 non-null   int64  
 5   Education                4410 non-null   int64  
 6   EducationField           4410 non-null   object 
 7   EmployeeCount            4410 non-null   int64  
 8   EmployeeID               4410 non-null   int64  
 9   Gender                   4410 non-null   object 
 10  JobLevel                 4410 non-null   int64  
 11  JobRole                  4410 non-null   object 
 12  MaritalStatus            4410 non-null   object 
 13  MonthlyIncome            4410 non-null   int64  
 14  NumCompaniesWorked      

In [ ]:
path1 = '/content/M6-Data-Engineering-and-MLOps/employee_survey_data.csv'

In [ ]:
ES = pd.read_csv(path1)

In [ ]:
path2 = '/content/M6-Data-Engineering-and-MLOps/manager_survey_data.csv'

In [ ]:
MS = pd.read_csv(path2)

In [ ]:
EMS = pd.merge(ES, MS)

In [ ]:
EMS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4410 entries, 0 to 4409
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EmployeeID               4410 non-null   int64  
 1   EnvironmentSatisfaction  4385 non-null   float64
 2   JobSatisfaction          4390 non-null   float64
 3   WorkLifeBalance          4372 non-null   float64
 4   Age                      4410 non-null   int64  
 5   Attrition                4410 non-null   object 
 6   BusinessTravel           4410 non-null   object 
 7   Department               4410 non-null   object 
 8   DistanceFromHome         4410 non-null   int64  
 9   Education                4410 non-null   int64  
 10  EducationField           4410 non-null   object 
 11  EmployeeCount            4410 non-null   int64  
 12  Gender                   4410 non-null   object 
 13  JobLevel                 4410 non-null   int64  
 14  JobRole                 

In [ ]:
hr_df = pd.merge(HR, EMS)

## **EDA**

In the second phase of the project there will be some EDA. Where data will be resized and reshaped for the machine learning purpose. The HR dataset have some missing values and some unsupported conditions for machine learning project. In this section managing data through EDA will be the key.

In [ ]:
hrdf = hr_df.drop_duplicates(subset = ['Age', 'Attrition', 'BusinessTravel', 'Department', 'DistanceFromHome', 'Education', 'EducationField', 'Gender', 'JobLevel', 'JobRole', 'MaritalStatus', 'MonthlyIncome', 'NumCompaniesWorked', 'PercentSalaryHike', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager'])

In [ ]:
hrdf.shape

(1498, 27)

In [ ]:
hrdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1498 entries, 0 to 4409
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      1498 non-null   int64  
 1   Attrition                1498 non-null   object 
 2   BusinessTravel           1498 non-null   object 
 3   Department               1498 non-null   object 
 4   DistanceFromHome         1498 non-null   int64  
 5   Education                1498 non-null   int64  
 6   EducationField           1498 non-null   object 
 7   EmployeeCount            1498 non-null   int64  
 8   EmployeeID               1498 non-null   int64  
 9   Gender                   1498 non-null   object 
 10  JobLevel                 1498 non-null   int64  
 11  JobRole                  1498 non-null   object 
 12  MaritalStatus            1498 non-null   object 
 13  MonthlyIncome            1498 non-null   int64  
 14  NumCompaniesWorked      

In [ ]:
#Numerical Data

hrdf.select_dtypes('int64' ,'float64').columns

Index(['Age', 'DistanceFromHome', 'Education', 'EmployeeCount', 'EmployeeID',
       'JobLevel', 'MonthlyIncome', 'PercentSalaryHike', 'StandardHours',
       'StockOptionLevel', 'TrainingTimesLastYear', 'YearsAtCompany',
       'YearsSinceLastPromotion', 'YearsWithCurrManager'],
      dtype='object')

In [ ]:
#Categorical Data

categorical_cols = hrdf.select_dtypes('object').columns
categorical_cols

Index(['Attrition', 'BusinessTravel', 'Department', 'EducationField', 'Gender',
       'JobRole', 'MaritalStatus', 'Over18'],
      dtype='object')

In [ ]:
#missing data (NULL or Nan)
hrdf.isnull().sum()

Age                         0
Attrition                   0
BusinessTravel              0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeID                  0
Gender                      0
JobLevel                    0
JobRole                     0
MaritalStatus               0
MonthlyIncome               0
NumCompaniesWorked         19
Over18                      0
PercentSalaryHike           0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           9
TrainingTimesLastYear       0
YearsAtCompany              0
YearsSinceLastPromotion     0
YearsWithCurrManager        0
EnvironmentSatisfaction    11
JobSatisfaction             7
WorkLifeBalance            13
dtype: int64

In [ ]:
#Replacing value with fillna with the rounded up meamn value

hrdf.NumCompaniesWorked = hrdf.NumCompaniesWorked.fillna(3)
hrdf.EnvironmentSatisfaction = hrdf.EnvironmentSatisfaction.fillna(3)
hrdf.JobSatisfaction = hrdf.JobSatisfaction.fillna(3)
hrdf.WorkLifeBalance = hrdf.WorkLifeBalance.fillna(3)
hrdf.TotalWorkingYears = hrdf.TotalWorkingYears.fillna(11)

In [ ]:
hrdf['NumCompaniesWorked'] = hrdf['NumCompaniesWorked'].astype(int)
hrdf['TotalWorkingYears'] = hrdf['TotalWorkingYears'].astype(int)
hrdf['EnvironmentSatisfaction'] = hrdf['EnvironmentSatisfaction'].astype(int)
hrdf['JobSatisfaction'] = hrdf['JobSatisfaction'].astype(int)
hrdf['WorkLifeBalance'] = hrdf['WorkLifeBalance'].astype(int)

In [ ]:
#no missing data (NULL or Nan)
hrdf.isnull().sum()

Age                        0
Attrition                  0
BusinessTravel             0
Department                 0
DistanceFromHome           0
Education                  0
EducationField             0
EmployeeCount              0
EmployeeID                 0
Gender                     0
JobLevel                   0
JobRole                    0
MaritalStatus              0
MonthlyIncome              0
NumCompaniesWorked         0
Over18                     0
PercentSalaryHike          0
StandardHours              0
StockOptionLevel           0
TotalWorkingYears          0
TrainingTimesLastYear      0
YearsAtCompany             0
YearsSinceLastPromotion    0
YearsWithCurrManager       0
EnvironmentSatisfaction    0
JobSatisfaction            0
WorkLifeBalance            0
dtype: int64

In [ ]:
hrdf

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,8,0,1,6,1,0,0,3,4,2
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,...,8,1,6,3,5,1,4,3,2,4
2,32,No,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,...,8,3,5,2,5,0,3,2,2,1
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,...,8,3,13,5,8,7,5,4,4,3
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,...,8,2,9,2,6,0,4,4,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3818,28,Yes,Travel_Rarely,Human Resources,1,3,Medical,1,3819,Female,...,8,1,11,2,1,0,0,3,2,3
3910,41,No,Non-Travel,Sales,2,2,Medical,1,3911,Male,...,8,2,10,5,3,1,2,3,2,3
4226,36,No,Travel_Rarely,Research & Development,2,3,Life Sciences,1,4227,Male,...,8,0,5,2,1,0,0,2,4,3
4395,40,No,Travel_Rarely,Research & Development,2,3,Life Sciences,1,4396,Male,...,8,1,9,4,9,4,7,1,4,3


In [ ]:
hrdf['Attrition'].replace({'No' : '0'}, inplace=True)
hrdf['Attrition'].replace({'Yes' : '1'}, inplace=True)

In [ ]:
hrdf

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
0,51,0,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,8,0,1,6,1,0,0,3,4,2
1,31,1,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,...,8,1,6,3,5,1,4,3,2,4
2,32,0,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,...,8,3,5,2,5,0,3,2,2,1
3,38,0,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,...,8,3,13,5,8,7,5,4,4,3
4,32,0,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,...,8,2,9,2,6,0,4,4,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3818,28,1,Travel_Rarely,Human Resources,1,3,Medical,1,3819,Female,...,8,1,11,2,1,0,0,3,2,3
3910,41,0,Non-Travel,Sales,2,2,Medical,1,3911,Male,...,8,2,10,5,3,1,2,3,2,3
4226,36,0,Travel_Rarely,Research & Development,2,3,Life Sciences,1,4227,Male,...,8,0,5,2,1,0,0,2,4,3
4395,40,0,Travel_Rarely,Research & Development,2,3,Life Sciences,1,4396,Male,...,8,1,9,4,9,4,7,1,4,3


In [ ]:
hrdf['NumCompaniesWorked'] = hrdf['NumCompaniesWorked'].astype(int)
hrdf['TotalWorkingYears'] = hrdf['TotalWorkingYears'].astype(int)
hrdf['EnvironmentSatisfaction'] = hrdf['EnvironmentSatisfaction'].astype(int)
hrdf['JobSatisfaction'] = hrdf['JobSatisfaction'].astype(int)
hrdf['WorkLifeBalance'] = hrdf['WorkLifeBalance'].astype(int)
hrdf['Attrition'] = hrdf['Attrition'].astype(int)

In [ ]:
hrdf.drop(['BusinessTravel', 'EmployeeCount', 'Department',
            'StandardHours', 'Over18'], axis=1, inplace=True)
print('Size of Full dataset is: {}'.format(hrdf.shape))

Size of Full dataset is: (1498, 22)


In [ ]:
hrdf.drop(['EducationField', 'Gender',
            'JobRole', 'MaritalStatus'], axis=1, inplace=True)
print('Size of Full dataset is: {}'.format(hrdf.shape))

Size of Full dataset is: (1498, 18)


In [ ]:
hrdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1498 entries, 0 to 4409
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   Age                      1498 non-null   int64
 1   Attrition                1498 non-null   int64
 2   DistanceFromHome         1498 non-null   int64
 3   Education                1498 non-null   int64
 4   EmployeeID               1498 non-null   int64
 5   JobLevel                 1498 non-null   int64
 6   MonthlyIncome            1498 non-null   int64
 7   NumCompaniesWorked       1498 non-null   int64
 8   PercentSalaryHike        1498 non-null   int64
 9   StockOptionLevel         1498 non-null   int64
 10  TotalWorkingYears        1498 non-null   int64
 11  TrainingTimesLastYear    1498 non-null   int64
 12  YearsAtCompany           1498 non-null   int64
 13  YearsSinceLastPromotion  1498 non-null   int64
 14  YearsWithCurrManager     1498 non-null   int64
 15  Envi

# **MLflow**

Here Elasticnet is selected from scikit-learn. To perform Elasticnet at forst some evaluation metrics has been set. Where RMSE, MAE and R2 (R square test) will be the priority. Then data has been spilited for train and test focusing on the attrition in the data set. Where yes is 1 and No is 0. Added to that for the prediction MLflow has started to run. Where in the backend API MLflow will start to store the prediction of the model with the accuracy level and the result. In the final stage of the MLflow, logging parameter and metrics is the final task. Based on which MLflow API will perform the chart related analysis. After performing the code, all the model will shift to MLflow.


In the next step evaluation of every training model based on different parameter will give different RMSE, MAE and R2 score. The operator of the model then can identify which model is the best depending on the different parameter that has been used in the model. In this project 5 different set of training model has been evaluated.

In [ ]:
def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

In [ ]:
def train_model(alpha, l1_ratio):
    train, test = train_test_split(hrdf)
    train_x = train.drop(["Attrition"], axis=1)
    test_x = test.drop(["Attrition"], axis=1)
    train_y = train[["Attrition"]]
    test_y = test[["Attrition"]]

    with mlflow.start_run():
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        mlflow.sklearn.log_model(lr, "model")

In [ ]:
train_model(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.3616166487810568
  MAE: 0.2656781367553978
  R2: 0.027034221163376282


In [ ]:
train_model(0.4, 0.4)

Elasticnet model (alpha=0.400000, l1_ratio=0.400000):
  RMSE: 0.38637640670969414
  MAE: 0.2791860285818781
  R2: 0.027360726582005945


In [ ]:
train_model(0.3, 0.3)

Elasticnet model (alpha=0.300000, l1_ratio=0.300000):
  RMSE: 0.3718984876576651
  MAE: 0.26739443734979035
  R2: 0.03475778063079582


In [ ]:
train_model(0.2, 0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.32264115857199543
  MAE: 0.24015312427219485
  R2: -0.00512807140264071


In [ ]:
train_model(0.1, 0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.34266332557248314
  MAE: 0.2564769136824772
  R2: 0.03246642425771462


# **NGROK**

NGROK has been used to visualize the MLflow UI. Through NGROK the local host will able to showcase the different model through MLflow UI. Here 5000 Mlflow UI port has been used in this project.

In [ ]:
# run tracking UI in the background
get_ipython().system_raw("mlflow ui --backend-store-uri sqlite:///mlruns.db --port 5000 &")# run tracking UI in the background

In [ ]:
# run tracking UI in the background
get_ipython().system_raw("mlflow ui --port 5000 &")
## Step 3 - Installing pyngrok for remote tunnel access using ngrock.com

from pyngrok import ngrok
from getpass import getpass
# Terminate open tunnels if any exist
ngrok.kill()
## Step 4 - Login on ngrok.com and get your authtoken from https://dashboard.ngrok.com/auth
# Enter your auth token when the code is running
NGROK_AUTH_TOKEN = getpass('Enter the ngrok authtoken: ')
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

Enter the ngrok authtoken: ··········


MLflow Tracking UI: https://c5a1-34-105-95-157.ngrok-free.app
